## Imports

In [ ]:
from datasets import Dataset
from keras.utils.np_utils import to_categorical

## Constantes

In [ ]:
dir = "../../"
dir_data = "data/"
dir_rotulacao = "rotulacao/result/"

arquivo_result = "rotulacao_200_comentarios_facebook_08-03-2023.json" ## Verificar caso a caso

dir_dataset = "datasets/dataset_facebook/"## Verificar caso a caso
dataset_name = "dataset_concatenate_facebook.json" ## Verificar caso a caso

## Lendo Resultados

In [ ]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_rotulacao}{arquivo_result}")

In [ ]:
resultado_doccano

## Lendo Dataset

In [ ]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

In [ ]:
dataset

## Prepara Dados Resultados

In [ ]:
resultado_doccano.features

### Remover Colunas Desnecessárias (opcional)

In [ ]:
resultado_doccano = resultado_doccano.remove_columns(["id", "text", "Comments"])

### Arrumar Colunas de Label

In [ ]:
def arruma_label(data):
    valor = 0
    if "Neutro" in data["label"]:
        valor = 1
    elif "Positivo" in data["label"]:
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

In [ ]:
resultado_doccano.features

## Prepara Dados Dataset

In [ ]:
dataset.features

### Adiciona a coluna de "label"

In [ ]:
if "label" not in dataset.features:
    dataset = dataset.add_column("label", [None] * len(dataset))

In [ ]:
dataset.features

## Merge - Dataset & Result Doccano

### Convertendo result Doccano para Dict

In [ ]:
dict_resultado_doccano = dict({dado["index"] : dado["label"] for dado in resultado_doccano})

### Merge

In [ ]:
def merge(data):
    if data["index"] in dict_resultado_doccano.keys():
        data["label"] = dict_resultado_doccano[data["index"]]
    return data
dataset = dataset.map(merge)

## Salvando Dataset

In [ ]:
dataset.to_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")